# Part 7: Hierarchical Security Knowledge (RAPTOR)

## Learning Objectives

By the end of this notebook, you will:
1. Understand RAPTOR (Recursive Abstractive Processing for Tree-Organized Retrieval)
2. Build hierarchical knowledge structures for security content
3. Implement recursive summarization at multiple levels
4. Create multi-level embeddings and indexing
5. Retrieve at different abstraction levels
6. Navigate from high-level concepts to specific details
7. Organize MITRE ATT&CK and OWASP in hierarchies

## The Problem with Flat Document Retrieval

Traditional RAG treats all documents equally at one level. This has limitations for hierarchical knowledge:

### Example: MITRE ATT&CK Framework

**Natural hierarchy:**
```
Tactic: Credential Access
├── Technique: OS Credential Dumping (T1003)
│   ├── Sub-technique: LSASS Memory (T1003.001)
│   ├── Sub-technique: Security Account Manager (T1003.002)
│   └── Sub-technique: NTDS (T1003.003)
├── Technique: Brute Force (T1110)
│   ├── Sub-technique: Password Guessing (T1110.001)
│   └── Sub-technique: Password Cracking (T1110.002)
└── ...
```

**Problem with flat retrieval:**
- Query: "What are credential access techniques?" → May retrieve specific sub-techniques without tactical context
- Query: "How does LSASS dumping work?" → May miss the broader credential dumping context
- No way to navigate up/down the hierarchy
- No high-level summaries available

## Solution: RAPTOR

**RAPTOR** builds a tree of abstractions:

### Key Ideas:

1. **Recursive Summarization**: Create summaries at multiple levels
2. **Tree Organization**: Organize knowledge in a hierarchy
3. **Multi-Level Retrieval**: Query at the appropriate abstraction level
4. **Context Preservation**: Maintain parent-child relationships

### RAPTOR Tree Structure:

```
Level 0 (Original Documents):
[Doc1: LSASS details] [Doc2: SAM details] [Doc3: NTDS details] ...

        ↓ Cluster + Summarize

Level 1 (Technique Summaries):
[Summary: OS Credential Dumping methods] [Summary: Brute Force methods] ...

        ↓ Cluster + Summarize

Level 2 (Tactic Summary):
[Summary: Credential Access techniques overview]
```

### Benefits:

- ✅ High-level queries get summaries
- ✅ Specific queries get details
- ✅ Navigate hierarchy (drill down/up)
- ✅ Better context understanding
- ✅ Scalable to large knowledge bases

---
## 1. Environment Setup

In [ ]:
# Install additional dependencies
!pip install -q scikit-learn umap-learn

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from typing import List, Dict, Optional, Tuple
import numpy as np
from collections import defaultdict

# Clustering and dimensionality reduction
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap

# LangChain imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document

# Load environment variables
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found")
else:
    print("✅ OpenAI API key loaded")

In [ ]:
# Initialize embeddings and LLM
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Embeddings and LLM initialized")

In [ ]:
# Load our existing vector store
vectorstore = Chroma(
    collection_name="owasp_llm_top10",
    embedding_function=embeddings,
    persist_directory="../data/chroma_db"
)

# Get all documents from vector store
all_docs = vectorstore.similarity_search("", k=100)  # Get all documents

print("✅ Vector store loaded")
print(f"   Total documents: {len(all_docs)}")

---
## 2. RAPTOR Overview

### Algorithm Steps:

1. **Start with leaf nodes** (original documents)
2. **Cluster** similar documents
3. **Summarize** each cluster → creates parent nodes
4. **Embed** summaries
5. **Repeat** on parent nodes until tree converges

### Tree Node Structure:

```python
class TreeNode:
    content: str              # Text content or summary
    embedding: List[float]    # Vector embedding
    level: int                # 0=leaf, 1=first summary, etc.
    children: List[TreeNode]  # Child nodes
    parent: Optional[TreeNode] # Parent node
    metadata: Dict            # Original metadata
```

---
## 3. Building the RAPTOR Tree

Let's implement the core RAPTOR algorithm.

In [ ]:
class RAPTORNode:
    """Node in the RAPTOR tree."""
    
    def __init__(
        self,
        content: str,
        embedding: Optional[np.ndarray] = None,
        level: int = 0,
        metadata: Optional[Dict] = None,
        children: Optional[List['RAPTORNode']] = None
    ):
        self.content = content
        self.embedding = embedding
        self.level = level
        self.metadata = metadata or {}
        self.children = children or []
        self.parent = None
        
        # Set parent references for children
        for child in self.children:
            child.parent = self
    
    def __repr__(self):
        return f"RAPTORNode(level={self.level}, children={len(self.children)}, content_len={len(self.content)})"

print("✅ RAPTORNode class defined")

In [ ]:
def cluster_documents(
    embeddings_array: np.ndarray,
    n_clusters: Optional[int] = None,
    min_cluster_size: int = 2
) -> np.ndarray:
    """
    Cluster documents based on embeddings.
    
    Args:
        embeddings_array: Array of embeddings (n_docs, embedding_dim)
        n_clusters: Number of clusters (if None, auto-determine)
        min_cluster_size: Minimum documents per cluster
        
    Returns:
        Cluster labels for each document
    """
    n_docs = embeddings_array.shape[0]
    
    # Auto-determine number of clusters if not specified
    if n_clusters is None:
        # Use sqrt(n) as heuristic, bounded by reasonable limits
        n_clusters = max(2, min(int(np.sqrt(n_docs)), n_docs // min_cluster_size))
    
    # Ensure we have enough documents for clustering
    if n_docs < n_clusters * min_cluster_size:
        n_clusters = max(1, n_docs // min_cluster_size)
    
    if n_clusters <= 1:
        # Too few documents, return single cluster
        return np.zeros(n_docs, dtype=int)
    
    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embeddings_array)
    
    return labels

print("✅ Clustering function created")

In [ ]:
def summarize_cluster(
    documents: List[str],
    llm,
    max_length: int = 500
) -> str:
    """
    Create a summary of a cluster of documents.
    
    Args:
        documents: List of document texts to summarize
        llm: Language model
        max_length: Maximum summary length in words
        
    Returns:
        Summary text
    """
    # Combine documents
    combined = "\n\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(documents)])
    
    # Create summary prompt
    summary_prompt = ChatPromptTemplate.from_template(
        """You are summarizing security documentation for a hierarchical knowledge system.

Create a comprehensive summary of the following related security documents.
The summary should:
1. Capture the main themes and key concepts
2. Preserve important security details (vulnerabilities, mitigations, risks)
3. Be self-contained and understandable without the original documents
4. Be approximately {max_length} words

Documents:
{documents}

Summary:"""
    )
    
    prompt_value = summary_prompt.invoke({"documents": combined, "max_length": max_length})
    response = llm.invoke(prompt_value)
    
    return response.content

print("✅ Summarization function created")

In [ ]:
def build_raptor_tree(
    documents: List[Document],
    embeddings_model,
    llm,
    max_levels: int = 3,
    min_cluster_size: int = 2
) -> List[List[RAPTORNode]]:
    """
    Build a RAPTOR tree from documents.
    
    Args:
        documents: List of documents to organize
        embeddings_model: Embedding model
        llm: Language model for summarization
        max_levels: Maximum tree depth
        min_cluster_size: Minimum documents per cluster
        
    Returns:
        List of levels, each containing RAPTORNodes
    """
    print(f"\n🌲 Building RAPTOR tree from {len(documents)} documents...\n")
    
    # Level 0: Create leaf nodes from original documents
    print("📄 Level 0: Creating leaf nodes...")
    
    # Get embeddings for all documents
    texts = [doc.page_content for doc in documents]
    doc_embeddings = embeddings_model.embed_documents(texts)
    
    leaf_nodes = [
        RAPTORNode(
            content=doc.page_content,
            embedding=np.array(emb),
            level=0,
            metadata=doc.metadata
        )
        for doc, emb in zip(documents, doc_embeddings)
    ]
    
    print(f"   Created {len(leaf_nodes)} leaf nodes\n")
    
    # Build tree levels
    tree_levels = [leaf_nodes]
    current_nodes = leaf_nodes
    
    for level in range(1, max_levels + 1):
        print(f"📊 Level {level}: Clustering and summarizing...")
        
        # Check if we have enough nodes to continue
        if len(current_nodes) < min_cluster_size:
            print(f"   Too few nodes ({len(current_nodes)}), stopping tree construction\n")
            break
        
        # Get embeddings matrix
        embeddings_matrix = np.array([node.embedding for node in current_nodes])
        
        # Cluster nodes
        labels = cluster_documents(embeddings_matrix, min_cluster_size=min_cluster_size)
        n_clusters = len(set(labels))
        
        print(f"   Clustered {len(current_nodes)} nodes into {n_clusters} groups")
        
        # Create parent nodes by summarizing clusters
        parent_nodes = []
        
        for cluster_id in range(n_clusters):
            # Get nodes in this cluster
            cluster_indices = np.where(labels == cluster_id)[0]
            cluster_nodes = [current_nodes[i] for i in cluster_indices]
            
            if len(cluster_nodes) == 0:
                continue
            
            # Summarize cluster
            cluster_texts = [node.content for node in cluster_nodes]
            summary = summarize_cluster(cluster_texts, llm)
            
            # Embed summary
            summary_embedding = embeddings_model.embed_query(summary)
            
            # Create parent node
            parent_node = RAPTORNode(
                content=summary,
                embedding=np.array(summary_embedding),
                level=level,
                children=cluster_nodes,
                metadata={'cluster_id': cluster_id, 'n_children': len(cluster_nodes)}
            )
            
            parent_nodes.append(parent_node)
        
        print(f"   Created {len(parent_nodes)} parent nodes\n")
        
        # Add level to tree
        tree_levels.append(parent_nodes)
        current_nodes = parent_nodes
        
        # Stop if we've converged to a single node or too few nodes
        if len(parent_nodes) <= 1:
            print(f"✅ Tree converged at level {level}\n")
            break
    
    print(f"🌲 RAPTOR tree built with {len(tree_levels)} levels")
    for i, level in enumerate(tree_levels):
        print(f"   Level {i}: {len(level)} nodes")
    print()
    
    return tree_levels

print("✅ RAPTOR tree building function created")

---
## 4. Build RAPTOR Tree from Our Security Documents

In [ ]:
# Build RAPTOR tree
raptor_tree = build_raptor_tree(
    documents=all_docs,
    embeddings_model=embeddings,
    llm=llm,
    max_levels=2,  # Build 2 levels of summaries (3 total levels including leaves)
    min_cluster_size=2
)

In [ ]:
# Examine the tree structure
print("\n📊 RAPTOR Tree Structure:\n")
print("="*80)

for level_idx, level_nodes in enumerate(raptor_tree):
    print(f"\nLevel {level_idx}: {len(level_nodes)} nodes")
    print("-"*80)
    
    for i, node in enumerate(level_nodes[:3]):  # Show first 3 nodes per level
        print(f"\nNode {i+1}:")
        print(f"  Content preview: {node.content[:150]}...")
        print(f"  Children: {len(node.children)}")
        if node.metadata:
            print(f"  Metadata: {node.metadata}")
    
    if len(level_nodes) > 3:
        print(f"\n... and {len(level_nodes) - 3} more nodes")

print("\n" + "="*80)

---
## 5. Hierarchical Retrieval Strategies

Now we can query at different abstraction levels.

In [ ]:
def raptor_retrieval(
    query: str,
    tree_levels: List[List[RAPTORNode]],
    embeddings_model,
    strategy: str = 'auto',
    k: int = 3
) -> List[RAPTORNode]:
    """
    Retrieve from RAPTOR tree using different strategies.
    
    Args:
        query: User query
        tree_levels: RAPTOR tree levels
        embeddings_model: Embedding model
        strategy: 'leaves', 'summaries', 'top', 'auto', 'all_levels'
        k: Number of nodes to retrieve
        
    Returns:
        List of retrieved RAPTORNodes
    """
    # Embed query
    query_embedding = np.array(embeddings_model.embed_query(query))
    
    # Select levels based on strategy
    if strategy == 'leaves':
        # Only search leaf nodes (original documents)
        search_levels = [0]
    elif strategy == 'summaries':
        # Only search summary nodes (level 1+)
        search_levels = list(range(1, len(tree_levels)))
    elif strategy == 'top':
        # Only search top level (most abstract)
        search_levels = [len(tree_levels) - 1]
    elif strategy == 'all_levels':
        # Search all levels
        search_levels = list(range(len(tree_levels)))
    else:  # 'auto'
        # Determine level based on query complexity
        # Simple heuristic: longer queries = more specific = search leaves
        query_words = query.split()
        if len(query_words) <= 5:
            # Short query = high-level = search summaries
            search_levels = [len(tree_levels) - 1, len(tree_levels) - 2] if len(tree_levels) > 1 else [0]
        else:
            # Long query = specific = search leaves and level 1
            search_levels = [0, 1] if len(tree_levels) > 1 else [0]
    
    # Remove invalid level indices
    search_levels = [l for l in search_levels if l < len(tree_levels)]
    
    print(f"\n🔍 RAPTOR Retrieval Strategy: {strategy}")
    print(f"   Searching levels: {search_levels}")
    
    # Collect all candidate nodes from selected levels
    candidates = []
    for level_idx in search_levels:
        candidates.extend(tree_levels[level_idx])
    
    print(f"   Total candidates: {len(candidates)}")
    
    # Compute similarity scores
    scored_nodes = []
    for node in candidates:
        similarity = np.dot(query_embedding, node.embedding) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(node.embedding)
        )
        scored_nodes.append((node, similarity))
    
    # Sort by similarity
    scored_nodes.sort(key=lambda x: x[1], reverse=True)
    
    # Return top k
    top_nodes = [node for node, score in scored_nodes[:k]]
    
    print(f"   Retrieved top {len(top_nodes)} nodes\n")
    
    return top_nodes

print("✅ RAPTOR retrieval function created")

---
## 6. Demonstrations

Let's test hierarchical retrieval with different query types.

### Example 1: High-Level Query

In [ ]:
high_level_query = "What are LLM security risks?"

print("="*80)
print(f"❓ High-Level Query: {high_level_query}")
print("="*80)

# Try different strategies
for strategy in ['top', 'auto', 'leaves']:
    print(f"\n{'─'*80}")
    print(f"Strategy: {strategy}")
    print(f"{'─'*80}")
    
    results = raptor_retrieval(
        query=high_level_query,
        tree_levels=raptor_tree,
        embeddings_model=embeddings,
        strategy=strategy,
        k=2
    )
    
    for i, node in enumerate(results, 1):
        print(f"\n{i}. Level {node.level} Node:")
        print(f"   Content: {node.content[:200]}...")
        print(f"   Children: {len(node.children)}")

### Example 2: Specific Query

In [ ]:
specific_query = "How do I prevent prompt injection attacks in my LLM application?"

print("="*80)
print(f"❓ Specific Query: {specific_query}")
print("="*80)

# Auto strategy should search leaves for specific queries
results = raptor_retrieval(
    query=specific_query,
    tree_levels=raptor_tree,
    embeddings_model=embeddings,
    strategy='auto',
    k=3
)

for i, node in enumerate(results, 1):
    print(f"\n{i}. Level {node.level} Node:")
    print(f"   Content: {node.content[:300]}...")
    if node.metadata.get('title'):
        print(f"   Title: {node.metadata['title']}")

---
## 7. Complete RAG with RAPTOR

In [ ]:
def rag_with_raptor(
    query: str,
    tree_levels: List[List[RAPTORNode]],
    embeddings_model,
    llm,
    strategy: str = 'auto',
    k: int = 3
) -> str:
    """
    Complete RAG pipeline using RAPTOR hierarchical retrieval.
    """
    print(f"\n{'='*80}")
    print(f"🌲 RAG with RAPTOR")
    print(f"{'='*80}\n")
    
    # Retrieve nodes
    nodes = raptor_retrieval(query, tree_levels, embeddings_model, strategy, k)
    
    # Format context
    context_parts = []
    for i, node in enumerate(nodes, 1):
        level_desc = "Detail" if node.level == 0 else f"Summary (Level {node.level})"
        context_parts.append(f"[{level_desc}]:\n{node.content}")
    
    context = "\n\n".join(context_parts)
    
    # Generate answer
    print("📝 Generating answer...\n")
    
    answer_prompt = ChatPromptTemplate.from_template(
        """You are an AI security expert assistant using hierarchical knowledge retrieval.

The context below includes both high-level summaries and detailed information retrieved from different abstraction levels.

Context:
{context}

User Question: {question}

Instructions:
1. Provide a comprehensive answer using the multi-level context
2. Start with high-level overview if summaries are present
3. Include specific details when available
4. Maintain logical flow from general to specific

Answer:"""
    )
    
    prompt_value = answer_prompt.invoke({"context": context, "question": query})
    response = llm.invoke(prompt_value)
    
    return response.content

print("✅ RAG with RAPTOR pipeline created")

In [ ]:
# Test complete RAG with RAPTOR
query = "What are the main security concerns for LLM applications?"

answer = rag_with_raptor(
    query=query,
    tree_levels=raptor_tree,
    embeddings_model=embeddings,
    llm=llm,
    strategy='auto',
    k=3
)

print("\n" + "="*80)
print("📄 ANSWER")
print("="*80)
print(answer)
print("\n" + "="*80)

---
## 8. Comparison: RAPTOR vs Flat Retrieval

In [ ]:
def compare_raptor_vs_flat(query: str, vectorstore, tree_levels, embeddings_model, llm):
    """
    Compare RAPTOR hierarchical retrieval vs flat vector search.
    """
    print("\n" + "="*80)
    print(f"❓ Query: {query}")
    print("="*80)
    
    # Flat retrieval
    print("\n1️⃣  FLAT VECTOR SEARCH (Baseline)")
    print("-"*80)
    flat_docs = vectorstore.similarity_search(query, k=3)
    print(f"Retrieved {len(flat_docs)} documents:\n")
    for i, doc in enumerate(flat_docs, 1):
        print(f"{i}. {doc.metadata.get('id')}: {doc.metadata.get('title')}")
        print(f"   Preview: {doc.page_content[:150]}...\n")
    
    # RAPTOR retrieval
    print("\n2️⃣  RAPTOR HIERARCHICAL RETRIEVAL")
    print("-"*80)
    raptor_nodes = raptor_retrieval(query, tree_levels, embeddings_model, strategy='auto', k=3)
    print(f"\nRetrieved {len(raptor_nodes)} nodes:\n")
    for i, node in enumerate(raptor_nodes, 1):
        level_desc = "Original" if node.level == 0 else f"Summary L{node.level}"
        print(f"{i}. [{level_desc}] {len(node.children)} children")
        print(f"   Content: {node.content[:150]}...\n")
    
    print("\n" + "="*80)
    print("📊 ANALYSIS")
    print("="*80)
    print("✅ Flat retrieval: Returns only original documents at one level")
    print("✅ RAPTOR: Can return summaries for high-level queries, details for specific queries")
    print("✅ RAPTOR: Better context through hierarchical organization")
    print("✅ RAPTOR: Navigate from overview to specifics")
    print("\n" + "="*80 + "\n")

print("✅ Comparison function created")

In [ ]:
# Run comparison
compare_raptor_vs_flat(
    query="What are LLM vulnerabilities?",
    vectorstore=vectorstore,
    tree_levels=raptor_tree,
    embeddings_model=embeddings,
    llm=llm
)

---
## 9. Summary and Key Takeaways

### What We Built

✅ Complete RAPTOR implementation:
1. **Hierarchical Tree Structure**: Multi-level organization
2. **Recursive Summarization**: Cluster + summarize at each level
3. **Multi-Level Embeddings**: Index at all abstraction levels
4. **Flexible Retrieval Strategies**: Auto, top, summaries, leaves, all_levels
5. **Complete RAG Pipeline**: End-to-end with RAPTOR
6. **Comparison Framework**: RAPTOR vs flat retrieval

### Core Concepts Learned

1. **Hierarchical Knowledge**: Organize information at multiple abstraction levels
2. **Recursive Processing**: Bottom-up tree construction
3. **Clustering**: Group similar documents automatically
4. **Abstractive Summarization**: Create higher-level representations
5. **Multi-Level Retrieval**: Query at appropriate abstraction

### Key Insights

**RAPTOR Benefits:**
- ↑↑ Better for hierarchical knowledge (MITRE ATT&CK, OWASP)
- ↑ Supports both high-level and specific queries
- ↑ Navigate from overview to details
- ↑ Better context understanding
- ✅ Essential for large, structured knowledge bases

**When to Use RAPTOR:**
- Knowledge has natural hierarchy
- Users need both overviews and details
- Large document collections (100+)
- Multi-level navigation required

**When NOT to Use RAPTOR:**
- Small document collections (<50)
- Flat knowledge structure
- Only specific queries (no high-level)
- Latency-sensitive (tree building is slow)

### Production Recommendations

1. **Pre-build trees offline**: Don't build on-demand
2. **Cache tree structure**: Save to disk/database
3. **Update incrementally**: Add new docs without full rebuild
4. **Tune clustering**: Experiment with cluster sizes
5. **Hybrid retrieval**: Combine RAPTOR with flat search
6. **Monitor tree quality**: Check summary coherence

### Next Steps

In **Part 8**, we'll implement **ColBERT**:
- Token-level embeddings
- Late interaction retrieval
- MaxSim scoring
- Better for technical content and code
- Precise matching for security patterns

Example: Finding similar code vulnerability patterns, matching exploit signatures.

---

### 🎯 Practice Exercises

1. **Implement Tree Persistence**: Save/load RAPTOR trees
2. **Add Metadata Propagation**: Preserve metadata up the tree
3. **Implement Tree Traversal**: Navigate parent → child
4. **Optimize Clustering**: Try different algorithms (HDBSCAN, Agglomerative)
5. **Build MITRE ATT&CK Tree**: Organize by tactics/techniques

### 📚 Further Reading

- [RAPTOR Paper](https://arxiv.org/abs/2401.18059)
- [Hierarchical Clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering)
- [Document Summarization](https://arxiv.org/abs/2304.07129)
- [Tree-Based Retrieval](https://arxiv.org/abs/2212.14024)